Model node and link IDs changed when rebuilding the base network from Ranch. By Conflating the old and new networks, we have a link ID crosswalk between the old link ID and new link ID. 

This notebook updates the IDs in project cards from old model IDs to new model IDs.

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy
import shutil

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters

from network_wrangler.utils import create_unique_shape_id

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# I/O

In [4]:
root_dir = "D:/metcouncil_network_rebuild"

old_network_project_card_folder_name = "ProjectCards"

new_network_project_card_folder_name = "ProjectCards_new_network"

old_project_cards_dir = os.path.join(root_dir, old_network_project_card_folder_name)
new_project_cards_dir = os.path.join(root_dir, new_network_project_card_folder_name)

In [5]:
# this notebook will copy the old project cards into the new project cards folder
# this notebook will directly overwrite the project cards

# check if the new folder exists
if os.path.isdir(new_project_cards_dir):
    print(f"Error: {new_project_cards_dir} exists! Please delete the existing folder.")
else:
    shutil.copytree(old_project_cards_dir, new_project_cards_dir)
    card_dir = new_project_cards_dir

In [6]:
lasso_dir = 'Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities'

In [7]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2023-05-30 16:10:20, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2023-05-30 16:10:20, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


In [76]:
# link id crosswalk file
crosswalk_file = os.path.join(root_dir, 'data', 'interim', 'model_network_id_crosswalk_step2_augmented.csv')

# Process

In [93]:
# read crosswalk

crosswalk_df = pd.read_csv(crosswalk_file)

In [94]:
# There are links in the old network that did not get match to the new network
# drop those links

crosswalk_df = crosswalk_df[crosswalk_df['model_link_id_new'].notnull()].copy()

In [95]:
# convert link id to int

crosswalk_df['model_link_id_new'] = crosswalk_df['model_link_id_new'].astype(int)

In [96]:
# create dictionary of old_id: [new_id_1, new_id_2,...]
# as there might be one-to-many, many-to-many matches

link_crosswalk = crosswalk_df.groupby(['model_link_id_old'])['model_link_id_new'].apply(list).reset_index()

link_crosswalk = dict(zip(link_crosswalk['model_link_id_old'], link_crosswalk['model_link_id_new']))

In [97]:
link_crosswalk

{1: [409420],
 3: [208243, 400359],
 4: [343056, 282309],
 6: [90253],
 7: [771172],
 8: [321093],
 9: [169687],
 12: [318570],
 13: [115644],
 14: [410498],
 18: [943181],
 19: [101107, 6733],
 20: [431453],
 22: [236240],
 23: [795383, 806949],
 24: [264002],
 26: [60108, 371141],
 27: [161715],
 28: [91509],
 29: [363994, 411025, 225533, 68168],
 30: [13626, 22626],
 31: [294239],
 32: [425469, 144409, 533327],
 33: [109665, 53465, 112902],
 34: [520562, 435266, 519095, 107600, 434610],
 35: [59698],
 36: [56906, 313618],
 37: [458138],
 38: [247868, 327341],
 39: [543978],
 40: [362872],
 41: [912396],
 42: [413793],
 43: [196420, 531270, 101137, 303830, 225306, 255161],
 44: [13561, 81857],
 45: [314642, 541995, 451583, 405177],
 46: [536301],
 47: [175556],
 48: [277118, 115190],
 49: [349043],
 50: [115809],
 51: [69529],
 52: [772671, 905580, 924557, 845581, 862004, 945340],
 53: [170357, 194705],
 55: [270359],
 56: [630577],
 57: [477531, 430658, 438941],
 58: [482574],
 59: 

# Update project cards with new IDs

In [14]:
# get a list of project card files

suffix = ["*.yaml", "*.yml"] #"*.wrangler", "*.wr"
card_list = []
for s in suffix:
    card_list.extend(glob.glob(os.path.join(card_dir, "**", s), recursive=True))

In [15]:
# print # project cards

print(str(len(card_list)) + " project cards")

101 project cards


In [37]:
import re

In [48]:
# define functions

def map_new_id(x, crosswalk):
    """
    return value from crosswalk
    """
    return crosswalk.get(x)

def transit_id_change(d, links_missing_correspondence_list):
    """
    update model node id in transit routing project cards
    """
    for p in d.get('properties'):
        if p.get('property') == 'routing':
            _existing = []
            _set = []
            for n in p.get('existing'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    print("Warning: old node id {} does not have correspondence".format(abs(n)))
                _existing.append(node)
            for n in p.get('set'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    print("Warning: old node id {} does not have correspondence".format(abs(n)))
                _set.append(node)
                
            p['existing'] = _existing
            p['set'] = _set

    return d

def roadway_id_change(d, links_missing_correspondence_list):
    """
    update model link and node id in roadway project cards
    """
    if d.get('category') in [
        'Roadway Property Change', 'Parallel Managed lanes'
    ]:
        
        if d.get('facility').get('link') == 'all':
            return d
        
        for link in d.get('facility').get('link'):
            if "model_link_id" in list(link.keys()):
                
                _model_link_id_list = []
                for l in link.get('model_link_id'):
                    if l in list(link_crosswalk.keys()):
                        #_model_link_id_list.append(
                        #    map_new_id(l, link_crosswalk)
                        #)
                        _model_link_id_list = _model_link_id_list + map_new_id(l, link_crosswalk)
                    else:
                        print("Warning: old link id {} does not have correspondence".format(abs(l)))
                        if l not in links_missing_correspondence_list:
                            links_missing_correspondence_list.append(l)

                link['model_link_id'] = _model_link_id_list
        
        return d
    
    if d.get('category') in ['Add New Roadway']:
        for link in d.get('links'):
            if link['A'] in list(node_crosswalk.keys()):
                link['A'] = map_new_id(link['A'], node_crosswalk)
            else:
                print("Warning: old node id {} does not have correspondence".format(abs(n)))
            if link['B'] in list(node_crosswalk.keys()):
                link['B'] = map_new_id(link['B'], node_crosswalk)
            else:
                print("Warning: old node id {} does not have correspondence".format(abs(n)))
        
        return d
    
    if d.get('category') in ['Roadway Deletion']:
        _model_link_id_list = []
        for l in d.get('links').get('model_link_id'):
            if l in list(link_crosswalk.keys()):
                link = map_new_id(l, link_crosswalk)

            else:
                print("Warning: old link id {} does not have correspondence".format(abs(l)))
                if l not in links_missing_correspondence_list:
                    links_missing_correspondence_list.append(l)
            #_model_link_id_list.append(link)
            _model_link_id_list = _model_link_id_list + link

        d['links']['model_link_id'] = _model_link_id_list
        
        return d
        

def update_id(card, links_missing_correspondence_list):
    """
    check if it's transit or highway project card
    """
    for change in card.changes:
        if change.get('category') == 'Transit Service Property Change':
            change = transit_id_change(change, links_missing_correspondence_list)

        else:
            change = roadway_id_change(change, links_missing_correspondence_list)

    return card, links_missing_correspondence_list

def update_project_cards(
    path,
):
    """
    main function
    """
    suffix = ["*.yaml", "*.yml"]
    card_list = []
    for s in suffix:
        card_list.extend(glob.glob(os.path.join(path, "**",s), recursive = True))
    print(str(len(card_list)) + " project cards")

    failed_df = pd.DataFrame()
        
    for file in card_list:
        card = ProjectCard.read(
            file,
            validate = False
        )

        links_missing_correspondence_list = []

        card, links_missing_correspondence_list = update_id(card, links_missing_correspondence_list)

        failed_df = pd.concat(
            [
                failed_df,
                pd.DataFrame(
                    data = {
                        "project_card_file_name": file, 
                        "old_model_id": links_missing_correspondence_list
                    }
                )
            ],
            sort = False,
            ignore_index = True
        )
        
        card.__dict__.pop('file', None)
        card.__dict__.pop('valid', None)
        
        card.write(
            filename = file
        )

    failed_df.drop_duplicates(inplace = True)
    
    return failed_df

In [98]:
# call main function
# All project cards currently refer to link ids only

links_failed_to_get_new_id_df = pd.DataFrame()

links_failed_to_get_new_id_df = update_project_cards(path = card_dir)

101 project cards
2023-05-31 15:29:36, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp1.yml
2023-05-31 15:29:36, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp1.yml
2023-05-31 15:29:38, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp15.yml
2023-05-31 15:29:38, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp15.yml
2023-05-31 15:29:39, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp2.yml
2023-05-31 15:29:39, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp2.yml
2023-05-31 15:29:41, INFO: Wrote project card to: D:/metcouncil_network_rebuild\ProjectCards_new_network\BaseCorrections\AsgnGrp4.yml
2023-05-31 15:29:41, INFO: Wrote project c

In [101]:
links_failed_to_get_new_id_df['old_model_id'] = links_failed_to_get_new_id_df['old_model_id'].astype(int)

In [102]:
# write out old links ids in project cards that did not have a new link id correspondence
# use might need to manually check and add those correspondence

links_failed_to_get_new_id_df.to_csv(os.path.join(card_dir, "missing_links_in_new_cards.csv"), sep=',',index=False)